In [1]:
import tensorflow as tf

In [2]:
vocab_size = 20000
max_len = 200

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.imdb.load_data(num_words=vocab_size)
print(f'학습 리뷰 개수 : {len(x_train)}')
print(f'검증 리뷰 개수 : {len(x_test)}')

17464789/17464789 [==============================] - 2s 0us/step
학습 리뷰 개수 : 25000
검증 리뷰 개수 : 25000


In [3]:
x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train, maxlen=max_len)
x_test = tf.keras.preprocessing.sequence.pad_sequences(x_test, maxlen=max_len)

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Embedding

In [5]:
embedding_dim = 32
hidden_units = 32

rnn_model = Sequential()
rnn_model.add(Embedding(vocab_size, embedding_dim))
rnn_model.add(SimpleRNN(hidden_units))
rnn_model.add(Dense(1, activation='sigmoid'))

rnn_model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

In [6]:
rnn_history = rnn_model.fit(x_train, y_train, epochs=5, batch_size=64, validation_split=0.2)

Epoch 1/5
313/313 [==============================] - 6s 16ms/step - loss: 0.5442 - acc: 0.7147 - val_loss: 0.4367 - val_acc: 0.8044
Epoch 2/5
313/313 [==============================] - 5s 16ms/step - loss: 0.3442 - acc: 0.8580 - val_loss: 0.3365 - val_acc: 0.8550
Epoch 3/5
313/313 [==============================] - 5s 16ms/step - loss: 0.2676 - acc: 0.8968 - val_loss: 0.4016 - val_acc: 0.8258
Epoch 4/5
313/313 [==============================] - 5s 16ms/step - loss: 0.2080 - acc: 0.9196 - val_loss: 0.5650 - val_acc: 0.7440
Epoch 5/5
313/313 [==============================] - 5s 16ms/step - loss: 0.1549 - acc: 0.9439 - val_loss: 0.4691 - val_acc: 0.8426


In [9]:
rnn_model.evaluate(x_test, y_test)

782/782 [==============================] - 2s 3ms/step - loss: 0.5022 - acc: 0.8312


[0.5022233128547668, 0.8311600089073181]

In [41]:
from tensorflow.keras.layers import Dropout, GlobalAveragePooling1D
from modules.encoder import Encoder
from modules.embedding import embedding
from modules.positional_encoder import positional_encoding

In [76]:
class Classifier(tf.keras.Model):
    def __init__(self, d_model, num_heads, dropout, d_ff):
        super(Classifier, self).__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.dropout = dropout
        self.d_ff = d_ff
        
        self.transformer_encoder = Encoder(d_model=d_model, num_heads=num_heads, dropout=dropout, d_ff=d_ff)
        self.embedding_layer = Embedding(vocab_size, d_model)
        self.dropout_layer = Dropout(dropout)
        self.pooling_layer = GlobalAveragePooling1D()
        self.output_layer = Dense(1, activation='sigmoid')
    
    def call(self, inputs):
        embedding_vector = embedding(embedding_layer=self.embedding_layer,
                                     inputs=inputs,
                                     d_model=self.d_model)
        embedding_vector = positional_encoding(dropout_layer=self.dropout_layer,
                                               inputs=embedding_vector,
                                               position=max_len,
                                               d_model=self.d_model)
        encoded_vector = self.transformer_encoder.encode(inputs=embedding_vector)
        encoded_vector = self.pooling_layer(encoded_vector)
        return self.output_layer(encoded_vector)

In [77]:
transformer_model = Classifier(d_model=embedding_dim, num_heads=2, dropout=0.1, d_ff=hidden_units)
transformer_model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

In [78]:
transformer_history = transformer_model.fit(x_train, y_train, epochs=5, batch_size=64, validation_split=0.2)

Epoch 1/5
313/313 [==============================] - 17s 41ms/step - loss: 0.5744 - acc: 0.7345 - val_loss: 0.4199 - val_acc: 0.8262
Epoch 2/5
313/313 [==============================] - 11s 36ms/step - loss: 0.3603 - acc: 0.8499 - val_loss: 0.3288 - val_acc: 0.8586
Epoch 3/5
313/313 [==============================] - 12s 37ms/step - loss: 0.2883 - acc: 0.8805 - val_loss: 0.3077 - val_acc: 0.8722
Epoch 4/5
313/313 [==============================] - 11s 36ms/step - loss: 0.2525 - acc: 0.8963 - val_loss: 0.2955 - val_acc: 0.8788
Epoch 5/5
313/313 [==============================] - 11s 35ms/step - loss: 0.2235 - acc: 0.9132 - val_loss: 0.2913 - val_acc: 0.8846


In [79]:
transformer_model.evaluate(x_test, y_test)

782/782 [==============================] - 3s 4ms/step - loss: 0.2922 - acc: 0.8769


[0.2921871244907379, 0.8769199848175049]